In [ ]:
import tailer
import re
import threading
import csv
import time

def monitor_log_file(log_file_path, patterns, alert_message):
    """
    Monitor a log file for new entries matching any of the specified patterns and alert when such an entry is found.
    Prints the matching string along with the alert message.
    """
    # Compile the patterns outside the loop for efficiency
    compiled_patterns = [re.compile(pattern) for pattern in patterns]
    while True:
        try:
            # Open the log file and monitor for new lines
            with open(log_file_path, "r") as file:
                # Let me know it's found the file
                print(f"{log_file_path} found!")
                for line in tailer.follow(file):
                    # Check each pattern for a match in the current line
                    for compiled_pattern in compiled_patterns:
                        match = compiled_pattern.search(line)
                        if match:
                            # Print the alert with the matching part of the line
                            print(f"{alert_message} {match.group()}")
                            break  # Exit the loop after the first match to avoid multiple alerts for the same line
        except FileNotFoundError:
            print(f"File {log_file_path} not found, checking again in 5 seconds.")
            time.sleep(5)  # Wait for some time before checking again, e.g., 10 seconds

def monitor_csv_file(csv_file_path, patterns, alert_message, column_index=0):
    # Compile regex patterns outside the loop for efficiency
    compiled_patterns = [re.compile(pattern) for pattern in patterns]
    #unit_pattern = re.compile(r" LOC_UNIT_([A-Z]*)_NAME")  # Compile this pattern too for efficiency
    new_barb_unit_pattern = re.compile(r" UNIT_([A-Z]*)") # Barbarians.csv string when a new one is created.

    while True:
        try:
            # Open the CSV file for reading
            with open(csv_file_path, "r") as file:
                # Let me know it's found the file
                print(f"{csv_file_path} found!")
                # Use tailer.follow to monitor the file for new lines
                for line in tailer.follow(file):
                    # Process the latest line from the CSV file
                    reader = csv.reader([line])
                    for row in reader:
                        if len(row) > column_index:
                            # Check each compiled pattern for a match
                            for compiled_pattern in compiled_patterns:
                                if compiled_pattern.search(row[column_index]):
                                    # If a pattern matches, search for the specific unit string
                                    # Check for matches with the new pattern in addition to the existing one
                                    new_barb_string = new_barb_unit_pattern.search(row[column_index])
                                    new_barb_coords = row[column_index + 1]
                                    if new_barb_string:
                                        unit_string = new_barb_string.group(1)
                                        print(f"{alert_message} {unit_string} at{new_barb_coords}")
                                    # matching_string = unit_pattern.search(row[column_index])
                                    # if matching_string:
                                    #     unit_string = matching_string.group(1)
                                    #     print(f"{alert_message} {unit_string}")
                                    #     # Break after the first match to avoid multiple alerts for the same line
        except FileNotFoundError:
            print(f"File {csv_file_path} not found, checking again in 5 seconds.")
            time.sleep(5)  # Wait for some time before checking again, e.g., 10 seconds

def start_monitoring(files_info):
    """
    Start monitoring multiple files in separate threads, allowing for multiple patterns per file.
    Prints the specific string that triggered the alert.
    """
    threads = []

    for file_info in files_info:
        # Extract common parameters
        file_path = file_info['path']
        patterns = file_info['pattern']
        alert_message = file_info['message']

        # Determine the monitoring function and its arguments based on file type
        if file_info.get('type') == 'log':
            thread = threading.Thread(target=monitor_log_file, args=(file_path, patterns, alert_message))
        elif file_info.get('type') == 'csv':
            # Use .get() for 'column_index' to provide a default value if it's not specified
            column_index = file_info.get('column_index', 0)
            thread = threading.Thread(target=monitor_csv_file, args=(file_path, patterns, alert_message, column_index))
        else:
            print(f"Unsupported file type for {file_path}")
            continue  # Skip unsupported file types
        
        threads.append(thread)
        thread.start()

# Example usage
files_info = [
    #{'type': 'log', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\GameCore.log', 'pattern': r"ERROR", 'message': "Error found"},
    #{'type': 'log', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians_Units.csv', 'pattern': r"ERROR", 'message': "Error found"},
    {'type': 'csv', 'path': 'C:\\Users\\User\\AppData\\Local\\Firaxis Games\\Sid Meier\'s Civilization VI\\Logs\\Barbarians.csv', 'pattern': [r" UNIT_SPEARMAN",
                                                                                                                                                     r" UNIT_SCOUT",
                                                                                                                                                       r" UNIT_WARRIOR",
                                                                                                                                                         r" UNIT_BARBARIAN_HORSEMAN",
                                                                                                                                                         r" UNIT_GALLEY",
                                                                                                                                                         r" UNIT_QUADRIREME",
                                                                                                                                                         r" UNIT_SWORDSMAN",
                                                                                                                                                         r" UNIT_ARCHERE",
                                                                                                                                                         r" UNIT_MAN_AT_ARMSE",
                                                                                                                                                         r" UNIT_CROSSBOWMAN",
                                                                                                                                                         ], 'message': "New Barbarian Military Unit spawned:", 'column_index': 2}
]

start_monitoring(files_info)
